In [1]:
from os import walk
from ffprobe import FFProbe
import ffmpeg
import sys
import pathlib
from pathlib import Path

In [2]:
#create  a class to store the result for saving result
class Report:
    def __init__(self, file):
        #file path
        self.file = file
        self.formatokFile = ''
        self.report = ''
        self.formatC = False
        self.vCodec = False
        self.aCodec = False
        self.frameRate = False
        self.aRatio = False
        self.resolution = False
        self.vBitRate = False
        self.aBitRate = False
        self.aChannel = False
        
    def check(self):
        return(self.formatC and
              self.vCodec and
              self.aCodec and
              self.frameRate and
              self.aRatio and
              self.resolution and
              self.vBitRate and
              self.aBitRate and
              self.aChannel)

In [3]:
#Use to check if the file fulfils all requirements
def fileCheck(file, filename):
    #create new report
    newReport = Report(str(file))
    
    audioStream = None
    videoStream = None
    
    reportTXT = "Problems with file " + filename + ": \n"
    
    if ".mp4" in filename:
        newReport.formatC = True
    else:
        reportTXT += "Wrong video container format. Format container should be mp4 but it is " + filename.split(".")[1] + ".\n"
        
    #probe the file
    metaData = FFProbe(str(file))
    streams = metaData.streams
    
    #get the audio and video stream
    for s in streams:
        if s.is_audio() == True:
            audioStream = s
        elif s.is_video() == True:
            videoStream = s
            
            
    #check video stream is present
    if videoStream == None:
        reportTXT += "No video stream. \n"
        
    else:
        #check video codec
        if videoStream.codec() != "h264":
            reportTXT += "Wrong video codec. Video codec should be h.264 but it is " + str(videoStream.codec()) + ".\n"
        else:
            newReport.vCodec = True
            
        #check framerate
        fps = round(videoStream.frames() / videoStream.duration_seconds())
        if fps != 25:
            reportTXT += "Wrong frame rate. Frame rate should be 25 but it is " + str(fps) + ".\n"
        else:
            newReport.frameRate = True
            
        #check aspect ratio
        resolution = videoStream.frame_size()
        aRatio = resolution[0] / resolution[1]
        if aRatio != 16/9:
            reportTXT += "Wrong aspect ratio. Aspect ratio should be 16/9 or " + str(16/9) + " but it is " + str(aRatio) + "\n"
        else:
            newReport.aRatio = True
        
        #check resolution
        if resolution != (640, 360):
            reportTXT += "Wrong resolution. Resolution should be 640x360 but it is " + str(resolution[0]) + "x" + str(resolution[1]) + ".\n"
        else:
            newReport.resolution = True
        
        #check video bit rate
        if videoStream.bit_rate == "N/A":
            reportTXT += "No video bit rate.\n"
        else:
            vBitRate = int(videoStream.bit_rate) / 1000000
            if (vBitRate < 2 or vBitRate > 5):
                reportTXT += "Wrong video bit rate. Video bit rate should be between 2-5 mbps but it is " + str(vBitRate) + " mbps.\n"
            else:
                newReport.vBitRate = True
        
    
    #check if audio stream is present
    if audioStream == None:
        reportTXT += "No audio stream \n"
    
    else:
        #check audio codec
        if audioStream.codec() != "aac":
            reportTXT += "Wrong audio codec. Audio codec should be aac but it is " + str(audioStream.codec()) + " .\n"
            
        else:
            newReport.aCodec = True
        
        #check audio bit rate
        aBitRate = int(audioStream.bit_rate) / 1000
        if aBitRate > 256:
            reportTXT += "Wrong audio bit rate. Audio bit rate should be up to 256 kbps but it is " + str(aBitRate) + " kbps.\n"
        else:
            newReport.aBitRate = True
            
        #check audio channels
        if audioStream.channel_layout != "stereo":
            reportTXT += "Wrong audio channel. Audio channels should be stereo but it is " + audioStream.channel_layout + ".\n"
        else:
            newReport.aChannel = True
            
        reportTXT += "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n\n\n"
    newReport.report = reportTXT
    return newReport

In [4]:
#function to fix the file if it fails the requirements
def fileFix(report):
    #we check if there is any errors if there are none, we return
    if report.check() == True:
        return
    
    #create new file with format ok
    name = report.file.split(".")[0]
    newName = name + "_formatOK" + ".mp4"
    report.formatokFile = newName
    
    #get the video and audio streams to change
    streams = ffmpeg.input(report.file)
    video = streams.video
    audio = streams.audio
    
    #change resolution, frame rate and size
    video = video.filter("scale", width="640", height="360")
    video = video.filter("fps", fps=25)
    
    #change the other parameters
    output = ffmpeg.output(audio, video, report.formatokFile, vcodec="libx264", acodec="aac", video_bitrate="3.5M", audio_bitrate="256K")
    
    #save and write the file
    output.overwrite_output().run()

In [5]:
#asset folder
assets = Path("Exercise_3_files")

#get all the files in the asset folder
files = next(walk("Exercise_3_files"), (None, None, []))[2]

#loop through all the filenames
for f in files:
    
    #check if there is any formatOK files and skip them
    if "_formatOK" in f:
        continue
        
    #create the path to the file
    file = Path(assets/f)
    
    #check format of file
    report = fileCheck(file, f)
    
    #open the report text file and append the report to it
    with open("report.txt", "a") as rt:
        rt.write(report.report)
        
    #check if there are any issues
    if report.check() == True:
        continue
    
    #fix file if there are issues
    fileFix(report)

In [6]:
#loop through all the files
for f in files:
    #we get the files that were changed and see if there are any issues even after the fix
    if "_formatOK" in f:
        #create the path to the file and check the file
        file = Path(assets/f)
        report = fileCheck(file, f)
        
        #we check if there was any issues
        if report.check() == True:
            continue
            
        #open the report text file and append the report to it if there are any issues
        with open("report.txt", "a") as rt:
            rt.write(report.report)